
# Heatmap_cam_FS


In [1]:


from tensorflow.python.keras.backend import clear_session
from tensorflow.python.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.python.keras.utils.layer_utils import print_summary
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.models import load_model, Model
import numpy as np
from resnet_convlstm import ResNet
from Utility_functions import create_sequence,FreezeBatchNormalization
from datetime import datetime as dt
from DFVDSequence import DFVDSequence
import matplotlib.pyplot as plt
print("yolo")
from sklearn.metrics import classification_report
import pandas as pd
# import shutil
from tensorflow.python.keras import layers
# from src.xception_convlstm import Xception
# print(cv2.__version__)
# import random
# random.seed(32)

dataset_dir='DeepFakeDatasetReal'
# from src.cl_basic import cl_basic
import ipykernel
import os
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import backend as K
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
import cv2
import matplotlib.pyplot as plt

# Choose GPU NUMBERS [0, 1, 2, 3]
os.environ["CUDA_VISIBLE_DEVICES"]="0"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)
sess.run(tf.global_variables_initializer())

In [2]:
model = load_model('models/FaceSwap/ResNet50(bk)/19-12-17-10_03/ResNet50(bk)_19-12-17-10_03-130-0.99.hdf5')

In [3]:
is_training = False
top_k_layers=120
model,df=FreezeBatchNormalization(is_training,top_k_layers,model)
adam_fine = Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
weights = model.layers[-1].get_weights()[0]
model2 = Model(inputs=model.inputs, outputs=[model.layers[-4].output, model.output])
model.compile(loss='binary_crossentropy',
                  optimizer=adam_fine,
                  metrics=['accuracy'])
print_summary(model, line_length=150, positions=None, print_fn=None)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df)

In [4]:
frames_per_video_per_batch=5
frames_per_video=80
image_size=240
test_video_per_batch=1
data_augmentation=False
training=False
X_DeepFake,y_DeepFake,class_weights_DeepFake=create_sequence(['datasets/FaceSwap/test/'],frames_per_video_per_batch,frames_per_video)
DeepFake_it=DFVDSequence(X_DeepFake,y_DeepFake,test_video_per_batch,frames_per_video_per_batch,image_size,data_augmentation,training)

In [10]:
image_size=(240,240)

it=DeepFake_it.__getitem__(1)

fig=plt.figure(figsize=(30, 100))
columns = 5
rows = 1
for i in range(5):
    it_0=it[0]
    weights = model.layers[-1].get_weights()[0]
    wx=(weights[:,0]).reshape(2048,1)
    [base_model_outputs, prediction] = model2.predict(it_0)

    # prediction = prediction[0][0]
    pred = np.argmax(prediction, axis=1)
#     print (prediction,pred)
    base_model_outputs = base_model_outputs[0][i]

#     print(prediction,base_model_outputs.shape,it_0[0,i].shape,wx.shape)
    fig.add_subplot(rows, columns, 1+i)
    if pred == 0:
        prediction=prediction[0][0]
        plt.title('Fake %.2f%%' % (prediction*100))
    else:
        prediction=prediction[0][1]
        plt.title('Real %.2f%%' % (prediction*100))

    cam = (prediction) * np.matmul(base_model_outputs, wx)

    cam -= cam.min()
    cam /= cam.max()
    cam -= 0.2
    cam /= 0.8
    img = cv2.resize(it_0[0,i], (it_0[0,i].shape[1], it_0[0,i].shape[0]))

    img = np.asarray(img, np.float64)
    cam = cv2.resize(cam, (240, 240))
    heatmap = cv2.applyColorMap(np.uint8(255*cam), cv2.COLORMAP_JET)
    heatmap=heatmap[:,:,::-1]
    heatmap[np.where(cam <= 0.00000001)] = 0
    out = cv2.addWeighted(img, 0.99, heatmap, 0.001, 0, dtype=cv2.CV_64F)
    plt.imshow(out)
plt.show()
print(it[1][0])